In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import torch

/home/zhiyuan/miniconda3/envs/MGTBench/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name_or_path = '/data1/models/distilbert-base-uncased'
from mgtbench import AutoDetector, AutoExperiment
from mgtbench.loading.dataloader import load
metric1 = AutoDetector.from_detector_name('LM-D', 
                                            model_name_or_path=model_name_or_path)
experiment = AutoExperiment.from_experiment_name('supervised',detector=[metric1])

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /data1/models/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
datatype = 'Essay'
llmname = 'ChatGLM'
data = load(datatype, llmname)
data['train']['text'] = data['train']['text'][:]
data['train']['label'] = data['train']['label'][:]
data['test']['text'] = data['test']['text'][:]
data['test']['label'] = data['test']['label'][:]

parsing data: 100%|██████████| 997/997 [00:00<00:00, 10470.72it/s]


In [5]:
len(data['train']['text']), data['train']['text'][0], data['train']['label'][0]

(1596,
 'It is important to note that psychopharmacology is a critical field of knowledge and expertise for any independent mental health practitioner. The main reason is the fact that psychopharmacology is a science-based study of medications and drugs on human psychology. It is becoming increasingly necessary for a mental health practitioner to be familiar and knowledgeable about pharmacological aspects of the treatment of mental issues.\nAs a future independent mental health practitioner, one must be fully equipped with skills and competencies in treating clients’ mental health problems, which is impossible without pharmacological interventions. It is stated that “even though psychiatrists represent the branch of medicine that specializes in psychopharmacology, they are directly responsible for providing only a fraction of professional services to the mentally ill” (Preston et al., 2021, p. 13). In other words, no treatment can be effective if the dedicated specialists are outsourci

In [6]:
experiment.load_data(data)

In [7]:
config = {'need_finetune': True,
          'need_save': False,
          'epochs': 1,
          'save_path': f'/data1/zzy/finetuned/bert_{datatype}_{llmname}' }

In [8]:
res = experiment.launch(**config)

Calculate result for each data point
Running prediction of detector LM-D


/home/zhiyuan/miniconda3/envs/MGTBench/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fine-tuning: 0 epoch: 100%|██████████| 100/100 [00:08<00:00, 11.93it/s]


Epoch [1/1], Loss: 0.00592301378871564
Fine-tune finished
Predict training data


100%|██████████| 1596/1596 [00:07<00:00, 206.32it/s]


Predict testing data


100%|██████████| 398/398 [00:01<00:00, 206.02it/s]

Run classification for results


In [9]:
res[0].train, res[0].test

(Metric(acc=0.9235588972431078, precision=0.8673913043478261, recall=1.0, f1=0.9289871944121071, auc=0.9800276066105112),
 Metric(acc=0.9271356783919598, precision=0.8728070175438597, recall=1.0, f1=0.9320843091334895, auc=0.9872225448852302))

In [10]:
torch.cuda.empty_cache()